In [10]:
import numpy as np
from astropy.io import fits
from marvin.tools import Maps
import marvin.utils.plot.colorbar as colorbar
from tqdm import tqdm
import matplotlib.pyplot as plt
#from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import scipy as scipy
from scipy.fft import fft
from scipy import ndimage

import multiprocessing
from multiprocessing import Pool
import platform
import copy

import Simpler_Classifier

from importlib import reload 


my_module = reload(Simpler_Classifier) # reload of Simpler_Classifier without restart

In [2]:
# New criteria:
# N [ivar > 0, mask NE 1, S/N>3] / N [ivar>0, mask Ne 1] >0.1
# Inclination > 60 -- through DAP topcat

HEADER_PATH = '/Users/runquanguan/Documents/bicone_training_set.fits'
hdul = fits.open(HEADER_PATH)
hdu = hdul[1].data
plateifu = hdu['PLATEIFU']


In [3]:
HEADER_PATH = '/Users/runquanguan/Documents/DAPALL_EXCL_INCL.fits'
hdul = fits.open(HEADER_PATH)
hdu = hdul[1].data
plateifu = hdu['PLATEIFU']


In [4]:
# good data:
data_good = ['8083-3704','8311-12704','8602-6101','9892-3703','12089-3703','12491-3704']

# bad data
data_bad = ['7981-12702', '8135-12701', '8244-3704', '8312-9101', '8462-12705', '8596-9102']

data = '7981-12702'

for i in data_bad:
    print(Simpler_Classifier.quality_test(i))

for i in data_good:
    print(Simpler_Classifier.quality_test(i))

In [5]:
def shell(data):
    try: 
        result = Simpler_Classifier.quality_test(data)
        return result
    except:
        pass


In [12]:
results = []

for data in tqdm(plateifu):
    try: 
        result = Simpler_Classifier.quality_test(data)
        results.append(result)
    except:
        pass

    

100%|██████████| 10737/10737 [9:47:57<00:00,  3.29s/it]   


if __name__ == '__main__':
    if platform.system() == "Darwin":
        multiprocessing.set_start_method('spawn')
        pool = mp.Pool(processes=6)
        results = []
        for result in pool.imap_unordered(shell, plateifu):
            results.append(result)
        pool.close()
        pool.join() 
        #print(results)

In [13]:
result_copy = copy.deepcopy(results)

In [14]:
clean = []
for i in result_copy:
    if i != None:
        clean.append(i)
len(clean)

10674

In [15]:
plateifu_l = []
snr_l = []
for data in clean:
    plateifu_l.append(data[0])
    snr_l.append(data[1])
    
    

In [37]:
c1 = fits.Column(name='plateifu', array=np.array(plateifu_l), format='20A')
c2 = fits.Column(name='SNR', array=np.array(snr_l), format='int16')
t = fits.BinTableHDU.from_columns([c1, c2])
t.writeto('DAPALL_SNR3.fits')

ValueError: invalid literal for int() with base 10: '0.9581151832460733'

In [17]:
miss_l = []
for i in plateifu:
    if i in plateifu_l:
        pass
    else:
        miss_l.append(i)
        
len(miss_l)

63

In [19]:
results2 = []

for data in tqdm(miss_l):
    try: 
        result2 = Simpler_Classifier.quality_test(data)
        results2.append(result2)
    except:
        pass

100%|██████████| 63/63 [08:34<00:00,  8.17s/it]


In [20]:
result_copy2 = copy.deepcopy(results2)

In [21]:
clean2 = []
for i in result_copy2:
    if i != None:
        clean2.append(i)
len(clean2)

61

In [22]:
for data in clean2:
    plateifu_l.append(data[0])
    snr_l.append(data[1])

In [24]:
len(plateifu_l)

10735

In [25]:
miss_l_2 = []
for i in plateifu:
    if i in plateifu_l:
        pass
    else:
        miss_l_2.append(i)
        
len(miss_l_2)

2

In [28]:
results3 = []

for data in tqdm(miss_l_2):
    try: 
        result3 = Simpler_Classifier.quality_test(data)
        results3.append(result2)
    except:
        pass

100%|██████████| 2/2 [00:11<00:00,  5.76s/it]


In [29]:
result3

('8982-1902', 1.0)

['11746-12704', '8982-1902']

In [31]:
miss_l_3 = '11746-12704'
result4 = Simpler_Classifier.quality_test(miss_l_3)
result4

[WARNING]: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray. (VisibleDeprecationWarning)


('11746-12704', 0.8626444159178434)

In [35]:
plateifu_l.append(result3[0])
snr_l.append(result3[1])

In [45]:
snr_new = []
for i in snr_l:
    snr_new.append(float(i))

In [50]:
c1 = fits.Column(name='plateifu', array=np.array(plateifu_l), format='20A')
c2 = fits.Column(name='SNR', array=np.array(snr_new), format='E')
t = fits.BinTableHDU.from_columns([c1, c2])
t.writeto('DAPALL_SNR3.fits')

In [47]:
def checkIfDuplicates_2(listOfElems):
    ''' Check if given list contains any duplicates '''    
    setOfElems = set()
    for elem in listOfElems:
        if elem in setOfElems:
            return True
        else:
            setOfElems.add(elem)         
    return False

In [48]:
checkIfDuplicates_2(plateifu_l)

False